In [41]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import re
from datetime import datetime
from tqdm import tqdm

# Directory containing the XML files
data_dir = "/Users/jerry/Desktop/fsan830spring2025/data/rawDataForTraining/pastPerformanceData"

# Get all XML files in the directory
xml_files = glob.glob(os.path.join(data_dir, "*.xml"))

dfs = []

def extract_date_from_filename(filename):
    # Example filename: SIMD20230502CD_USA.xml or similar
    # Try to extract an 8-digit date (YYYYMMDD) from the filename
    match = re.search(r'(\d{8})', os.path.basename(filename))
    if match:
        return match.group(1)
    else:
        return None

def format_yyyymmdd_to_iso(date_str):
    # Convert YYYYMMDD to YYYY-MM-DD
    try:
        return datetime.strptime(date_str, "%Y%m%d").strftime("%Y-%m-%d")
    except Exception:
        return None

def get_text(element, path, default=None):
    """Helper to get text from nested XML, returns default if not found."""
    found = element.find(path)
    if found is not None and found.text is not None:
        return found.text.strip()
    return default

def get_attrib(element, path, attrib, default=None):
    """Helper to get attribute from nested XML, returns default if not found."""
    found = element.find(path)
    if found is not None:
        return found.attrib.get(attrib, default)
    return default

# For more: collect all possible fields, including nested and attribute fields
# We'll also collect all <Horse> entries for each race, if present

for file in tqdm(xml_files, desc="Processing XML files"):
    try:
        tree = ET.parse(file)
        root = tree.getroot()
        # Extract date from filename
        file_date = extract_date_from_filename(file)
        race_date_iso = format_yyyymmdd_to_iso(file_date) if file_date else None
        # Each file is assumed to have one or more <Race> elements under the root
        for race in root.findall(".//Race"):
            race_data = {}

            # --- Basic race fields ---
            basic_fields = [
                ("RaceNumber", "RaceNumber"),
                ("DayEvening", "DayEvening"),
                ("Distance", "Distance/PublishedValue"),
                ("DistanceId", "Distance/DistanceId"),
                ("DistanceUnit", "Distance/DistanceUnit/Value"),
                ("AboutDistanceIndicator", "Distance/AboutDistanceIndicator"),
                ("RaceType", "RaceType/RaceType"),
                ("RaceTypeDesc", "RaceType/Description"),
                ("RestrictionType", "RestrictionType/Value"),
                ("RestrictionTypeDesc", "RestrictionType/Description"),
                ("SexRestriction", "SexRestriction/Value"),
                ("SexRestrictionDesc", "SexRestriction/Description"),
                ("AgeRestriction", "AgeRestriction/Value"),
                ("AgeRestrictionDesc", "AgeRestriction/Description"),
                ("PurseUSA", "PurseUSA"),
                ("Grade", "Grade"),
                ("Division", "Division"),
                ("ConditionText", "ConditionText"),
                ("ConditionsOfRace", "ConditionsOfRace"),
                ("PostTime", "PostTime"),
                ("CourseType", "Course/CourseType/Value"),
                ("CourseTypeDesc", "Course/CourseType/Description"),
                ("Surface", "Course/Surface/Value"),
                ("SurfaceDesc", "Course/Surface/Description"),
                ("BreedType", "BreedType/Value"),
                ("BreedTypeDesc", "BreedType/Description"),
                ("Track", "Track/Value"),
                ("TrackDesc", "Track/Description"),
                ("PurseCurrency", "PurseCurrency"),
                ("Country", "Country"),
                ("State", "State"),
                ("City", "City"),
                ("RaceName", "RaceName"),
                ("RaceClassification", "RaceClassification"),
                ("RaceStatus", "RaceStatus"),
                ("RaceSubType", "RaceSubType"),
                ("RaceSubTypeDesc", "RaceSubType/Description"),
                ("RaceSponsor", "RaceSponsor"),
                ("RaceGroup", "RaceGroup"),
                ("RaceGroupDesc", "RaceGroup/Description"),
                ("RaceDivision", "RaceDivision"),
                ("RaceDivisionDesc", "RaceDivision/Description"),
                ("RaceOrder", "RaceOrder"),
                ("RaceOrderDesc", "RaceOrder/Description"),
                ("RaceNotes", "RaceNotes"),
                ("RaceComments", "RaceComments"),
            ]
            for col, path in basic_fields:
                race_data[col] = get_text(race, path)
            race_data['RaceDate'] = race_date_iso  # Add RaceDate in YYYY-MM-DD format

            # --- Horse-level fields ---
            horses = race.findall(".//Horse")
            race_data['NumHorses'] = len(horses)

            # Define all horse-level fields to extract (field name, xml path)
            horse_fields = [
                ("HorseNames", "Name"),
                ("HorseNumbers", "ProgramNumber"),
                ("MorningLineOddsList", "MorningLineOdds"),
                ("Jockeys", "Jockey/Name"),
                ("Trainers", "Trainer/Name"),
                ("Owners", "Owner/Name"),
                ("Sires", "Sire/Name"),
                ("Dams", "Dam/Name"),
                ("HorseBirthYears", "BirthYear"),
                ("HorseSexes", "Sex"),
                ("HorseCountries", "Country"),
                ("HorseColors", "Color"),
                ("HorseWeights", "Weight"),
                ("HorseMedications", "Medication"),
                ("HorseEquipment", "Equipment"),
                ("HorseSilks", "Silks"),
                ("HorseEarnings", "Earnings"),
                ("HorseStarts", "Starts"),
                ("HorseWins", "Wins"),
                ("HorsePlaces", "Places"),
                ("HorseShows", "Shows"),
                ("HorseLastRaceDates", "LastRaceDate"),
                ("HorseLastRaceTracks", "LastRaceTrack"),
                ("HorseLastRaceFinish", "LastRaceFinish"),
                ("HorseLastRaceOdds", "LastRaceOdds"),
                ("HorseLastRaceJockeys", "LastRaceJockey"),
                ("HorseLastRaceTrainers", "LastRaceTrainer"),
                ("HorseLastRacePurse", "LastRacePurse"),
                ("HorseLastRaceClass", "LastRaceClass"),
                ("HorseLastRaceDistance", "LastRaceDistance"),
                ("HorseLastRaceSurface", "LastRaceSurface"),
                ("HorseLastRaceTime", "LastRaceTime"),
                ("HorseLastRaceComments", "LastRaceComments"),
                ("HorseLastRaceFirstCall", "LastRaceFirstCall"),
                ("HorseLastRaceSecondCall", "LastRaceSecondCall"),
                ("HorseLastRaceStretch", "LastRaceStretch"),
                ("HorseLastRaceBeatenLengths", "LastRaceBeatenLengths"),
                ("HorseLastRaceSpeedFigure", "LastRaceSpeedFigure"),
                ("HorseLastRaceClassRating", "LastRaceClassRating"),
                ("HorseLastRacePaceRating", "LastRacePaceRating"),
                ("HorseLastRaceWeightCarried", "LastRaceWeightCarried"),
                ("HorseLastRaceMedication", "LastRaceMedication"),
                ("HorseLastRaceEquipment", "LastRaceEquipment"),
                ("HorseLastRaceTrouble", "LastRaceTrouble"),
                ("HorseLastRaceWorkout", "LastRaceWorkout"),
                ("HorseLastRaceWorkoutTime", "LastRaceWorkoutTime"),
                ("HorseLastRaceWorkoutDistance", "LastRaceWorkoutDistance"),
                ("HorseLastRaceWorkoutSurface", "LastRaceWorkoutSurface"),
                ("HorseLastRaceWorkoutRank", "LastRaceWorkoutRank"),
                ("HorseLastRaceWorkoutDate", "LastRaceWorkoutDate"),
                ("HorseLastRaceWorkoutNotes", "LastRaceWorkoutNotes"),
            ]

            # For each horse-level field, collect all values for all horses and join with ";"
            for col, path in horse_fields:
                values = []
                for h in horses:
                    val = get_text(h, path)
                    if val:
                        values.append(val)
                race_data[col] = ";".join(values) if values else None

            # Add the file name for traceability
            race_data['SourceFile'] = os.path.basename(file)

            # --- More: Add even more fields if present in the XML ---
            # Example: Add all <Payout> entries for the race (if present)
            payouts = race.findall(".//Payout")
            payout_types = []
            payout_amounts = []
            for p in payouts:
                ptype = get_text(p, "Type")
                pamt = get_text(p, "Amount")
                if ptype:
                    payout_types.append(ptype)
                if pamt:
                    payout_amounts.append(pamt)
            race_data['PayoutTypes'] = ";".join(payout_types) if payout_types else None
            race_data['PayoutAmounts'] = ";".join(payout_amounts) if payout_amounts else None

            # Example: Add all <Scratch> entries for the race (if present)
            scratches = race.findall(".//Scratch")
            scratch_names = []
            for s in scratches:
                sname = get_text(s, "Name")
                if sname:
                    scratch_names.append(sname)
            race_data['ScratchedHorses'] = ";".join(scratch_names) if scratch_names else None

            # Example: Add all <StewardNote> entries for the race (if present)
            steward_notes = race.findall(".//StewardNote")
            notes = []
            for n in steward_notes:
                note = n.text.strip() if n.text else None
                if note:
                    notes.append(note)
            race_data['StewardNotes'] = ";".join(notes) if notes else None

            # Example: Add all <Weather> fields for the race (if present)
            weather = get_text(race, "Weather")
            race_data['Weather'] = weather

            # Example: Add all <TrackCondition> fields for the race (if present)
            track_condition = get_text(race, "TrackCondition")
            race_data['TrackCondition'] = track_condition

            # Example: Add all <RunUp> fields for the race (if present)
            runup = get_text(race, "RunUp")
            race_data['RunUp'] = runup

            # Example: Add all <Temp> fields for the race (if present)
            temp = get_text(race, "Temp")
            race_data['Temp'] = temp

            # Example: Add all <Wind> fields for the race (if present)
            wind = get_text(race, "Wind")
            race_data['Wind'] = wind

            # Example: Add all <Humidity> fields for the race (if present)
            humidity = get_text(race, "Humidity")
            race_data['Humidity'] = humidity

            # Example: Add all <Attendance> fields for the race (if present)
            attendance = get_text(race, "Attendance")
            race_data['Attendance'] = attendance

            # Example: Add all <Handle> fields for the race (if present)
            handle = get_text(race, "Handle")
            race_data['Handle'] = handle

            # Example: Add all <TotalHandle> fields for the race (if present)
            total_handle = get_text(race, "TotalHandle")
            race_data['TotalHandle'] = total_handle

            # Example: Add all <WagerType> fields for the race (if present)
            wager_types = []
            for w in race.findall(".//WagerType"):
                wtype = w.text.strip() if w.text else None
                if wtype:
                    wager_types.append(wtype)
            race_data['WagerTypes'] = ";".join(wager_types) if wager_types else None

            # Example: Add all <FractionalTimes> for the race (if present)
            fractional_times = []
            for ft in race.findall(".//FractionalTime"):
                ftime = ft.text.strip() if ft.text else None
                if ftime:
                    fractional_times.append(ftime)
            race_data['FractionalTimes'] = ";".join(fractional_times) if fractional_times else None

            # Example: Add all <SplitTimes> for the race (if present)
            split_times = []
            for st in race.findall(".//SplitTime"):
                stime = st.text.strip() if st.text else None
                if stime:
                    split_times.append(stime)
            race_data['SplitTimes'] = ";".join(split_times) if split_times else None

            # Example: Add all <SectionalTimes> for the race (if present)
            sectional_times = []
            for sec in race.findall(".//SectionalTime"):
                sectime = sec.text.strip() if sec.text else None
                if sectime:
                    sectional_times.append(sectime)
            race_data['SectionalTimes'] = ";".join(sectional_times) if sectional_times else None

            

            # Add the DataFrame for this race
            dfs.append(pd.DataFrame([race_data]))
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Concatenate all DataFrames
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
else:
    df_all = pd.DataFrame()

# Show more columns and rows for inspection
with pd.option_context('display.max_columns', 200, 'display.max_rows', 40):
    print(df_all)

df_all.head(10)



Processing XML files: 100%|██████████| 18/18 [00:02<00:00,  7.36it/s]

    RaceNumber DayEvening Distance DistanceId DistanceUnit  \
0            1          D       5F        500            F   
1            2          D       7F        700            F   
2            3          D       7F        700            F   
3            4          D   1 3/8M       1100            F   
4            5          D   6 1/2F        650            F   
..         ...        ...      ...        ...          ...   
174          7          D       6F        600            F   
175          8          D       7F        700            F   
176          9          D       6F        600            F   
177         10          D       1M        800            F   
178         11          D       6F        600            F   

    AboutDistanceIndicator RaceType           RaceTypeDesc RestrictionType  \
0                               MSW  MAIDEN SPECIAL WEIGHT            None   
1                               CLM               CLAIMING            None   
2                    

,RaceNumber,DayEvening,Distance,DistanceId,DistanceUnit,AboutDistanceIndicator,RaceType,RaceTypeDesc,RestrictionType,RestrictionTypeDesc,...,TrackSteward,TrackVet,TrackSecurity,TrackMedical,TrackAmbulance,TrackFire,TrackPolice,TrackFirstAid,TrackOtherStaff,TrackOtherNotes
0,1,D,5F,500,F,,MSW,MAIDEN SPECIAL WEIGHT,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2,D,7F,700,F,,CLM,CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3,D,7F,700,F,,CLM,CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
3,4,D,1 3/8M,1100,F,,ALW,ALLOWANCE,None,None,...,None,None,None,None,None,None,None,None,None,None
4,5,D,6 1/2F,650,F,,MCL,MAIDEN CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
5,6,D,1 1/16M,850,F,,CLM,CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
6,7,D,1 1/16M,850,F,,AOC,ALLOWANCE OPTIONAL CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
7,8,D,1M,800,F,,MSW,MAIDEN SPECIAL WEIGHT,None,None,...,None,None,None,None,None,None,None,None,None,None
8,1,D,1 1/16M,850,F,,CLM,CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None
9,2,D,6F,600,F,,CLM,CLAIMING,None,None,...,None,None,None,None,None,None,None,None,None,None


In [34]:
import glob

# Directory containing the XML files for results
results_dir = "/Users/jerry/Desktop/fsan830spring2025/data/rawDataForTraining/resultsData"
results_xml_files = glob.glob(os.path.join(results_dir, "*.xml"))

results_dfs = []

for file in results_xml_files:
    try:
        tree = ET.parse(file)
        root = tree.getroot()
        # Each file is assumed to have one or more <RACE> elements under the root
        for race in root.findall(".//RACE"):
            race_data = {}
            # Extract some basic fields (customize as needed)
            race_data['RaceNumber'] = race.attrib.get("NUMBER")
            race_data['Purse'] = race.findtext("PURSE")
            race_data['Distance'] = race.findtext("DISTANCE")
            race_data['TrackCondition'] = race.findtext("TRK_COND")
            race_data['RaceDate'] = root.attrib.get("RACE_DATE") if "RACE_DATE" in root.attrib else None
            # Add more fields as needed

            results_dfs.append(pd.DataFrame([race_data]))
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Concatenate all DataFrames
if results_dfs:
    results_df_all = pd.concat(results_dfs, ignore_index=True)
else:
    results_df_all = pd.DataFrame()

print(results_df_all)


    RaceNumber   Purse Distance TrackCondition    RaceDate
0            1   39000      850             SY  2023-05-20
1            2   54000      600             SY  2023-05-20
2            3   68000      750             MY  2023-05-20
3            4  120000      800             MY  2023-05-20
4            5   54000      850             FT  2023-05-20
..         ...     ...      ...            ...         ...
174          7  127000     1100             FM  2023-05-13
175          8   72000      800             FT  2023-05-13
176          9  134000      600             FT  2023-05-13
177         10  225000      550             FM  2023-05-13
178         11   63000      600             FT  2023-05-13

[179 rows x 5 columns]


In [35]:
join_df = pd.merge(df_all, results_df_all, on=['RaceNumber', 'RaceDate'], how='left')
join_df.head(10)

,RaceNumber,DayEvening,Distance_x,DistanceId,DistanceUnit,AboutDistanceIndicator,RaceType,RaceTypeDesc,RestrictionType,SexRestriction,...,ConditionsOfRace,PostTime,CourseType,CourseTypeDesc,Surface,SurfaceDesc,RaceDate,Purse,Distance_y,TrackCondition
0,1,D,5F,500,F,,MSW,MAIDEN SPECIAL WEIGHT,,,...,,5:00PM,D,Dirt,D,,2023-05-18,120000,500,FT
1,2,D,7F,700,F,,CLM,CLAIMING,,B,...,NW2 L,5:29PM,D,Dirt,D,,2023-05-18,31000,700,FT
2,3,D,7F,700,F,,CLM,CLAIMING,,,...,,5:58PM,D,Dirt,D,,2023-05-18,68000,700,FT
3,4,D,1 3/8M,1100,F,,ALW,ALLOWANCE,,B,...,NW1 X,6:27PM,T,Turf,T,,2023-05-18,127000,1100,FM
4,5,D,6 1/2F,650,F,,MCL,MAIDEN CLAIMING,,,...,,6:56PM,D,Dirt,D,,2023-05-18,50000,650,FT
5,6,D,1 1/16M,850,F,,CLM,CLAIMING,,,...,,7:25PM,D,Dirt,D,,2023-05-18,78000,850,FT
6,7,D,1 1/16M,850,F,,AOC,ALLOWANCE OPTIONAL CLAIMING,,B,...,NW2$ X,7:54PM,T,Turf,T,,2023-05-18,134000,850,FM
7,8,D,1M,800,F,,MSW,MAIDEN SPECIAL WEIGHT,,B,...,,8:23PM,D,Dirt,D,,2023-05-18,120000,800,FT
8,1,D,1 1/16M,850,F,,CLM,CLAIMING,,,...,,12:45PM,D,Dirt,D,,2023-05-20,39000,850,SY
9,2,D,6F,600,F,,CLM,CLAIMING,,,...,NW2 L,1:14PM,D,Dirt,D,,2023-05-20,54000,600,SY
